In [21]:
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import os
from dotenv import load_dotenv

# FMP News API Code

In [17]:
load_dotenv(r"api.env")

fmp_api_key = os.getenv("fmp_api_key")
if not fmp_api_key:
    raise ValueError("No API key set for fmp_api_key in .env file")

print("API key loaded successfully from .env file.")

API key loaded successfully from .env file.


In [20]:
stock_news = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

stock_news_rss_feed = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

press_releases = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

historical_social_sentiment = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

In [50]:
current_date =date.today()

for news in stock_news:
    url = f"https://financialmodelingprep.com/api/v3/stock_news?"

    page = 0
    limit = 1000
    all_news = []

    while True:
        params = {
        "apikey" : fmp_api_key,
        "tickers" : news['symbol'],
        "page": page,
        'from': news['ipo_date'],
        'to': current_date,
        'limit': limit
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {news['symbol']} on page {page}: {e}")
            break

        data = response.json()

        if not data:
            break


        all_news.extend(data)

        # if len(data) < limit:
        #     break

        page += 1

    csv_file = f"{news['symbol']}_news_data.csv"
    headers = ["Headline", "Brief", "URL", "Date"]
    formatted_data = []

    if all_news:
        for record in all_news:
            # if 'publishedDate' in record:
                # try:
                #     record['publishedDate'] = datetime.strptime(record['publishedDate'], '%Y-%m-%dT%H:%M:%S').strftime('%d-%m-%Y')
                # except ValueError as e:
                #     print(f"Error processing date {record['date']}: {e}")

            formatted_data.append({
            "Headline": record.get("title", ""),
            "Brief": record.get("text", ""),
            "URL": record.get("url", ""),
            "Date": record.get("publishedDate", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {news['symbol']} written to {csv_file} with {len(formatted_data)} articles!")
    else:
        print(f"No data found for {news['symbol']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

Data for AMZN written to AMZN_news_data.csv with 29053 articles!
Data for AAPL written to AAPL_news_data.csv with 26268 articles!
Data for GOOG written to GOOG_news_data.csv with 17556 articles!
Data for MSFT written to MSFT_news_data.csv with 16611 articles!
Data for META written to META_news_data.csv with 18242 articles!
Data for NVDA written to NVDA_news_data.csv with 18710 articles!
All stock data successfully written to their respective CSV files.


In [ ]:
# Load the existing CSV file
csv_filename = "news_data.csv"
df = pd.read_csv(csv_filename)

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score > 0.05:
        return "Positive"
    elif score < -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to each row
df["Sentiment"] = df.apply(lambda row: get_sentiment(f"{row['Headline']} {row['Brief']}"), axis=1)

# Save back to the same CSV file (overwrite with sentiment added)
df.to_csv(csv_filename, index=False)
print(f"Sentiment appended to {csv_filename} successfully!")